In [ ]:
import warnings
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

warnings.simplefilter("ignore")
%matplotlib inline

print(os.listdir("../input"))

# Load the training dataset

In [ ]:
df_train = pd.read_csv(os.path.join('..', 'input', 'train.csv'), index_col=0)

# Explore the training dataset

## Print row and types

In [ ]:
df_train.head()

In [ ]:
df_train.dtypes

## Check and remove duplicated rows

In [ ]:
df_train.duplicated().sum()

There is 7 duplicated rows, let's remove them

In [ ]:
df_train = df_train.drop_duplicates()
df_train.duplicated().sum()

## Check for null values

In [ ]:
df_train.isna().sum()

## Check for outliers

First let's describe the dataframe with pretty printed columns

In [ ]:
describe = df_train.describe()

def seconds_to_pretty(seconds):
    seconds = int(float(seconds))
    return '{} h {} m {} s ({} s)'.format(seconds // 3600, (seconds % 3600) // 60, (seconds % 3600) % 60, seconds)

describe = describe.drop('count') # to disable scientific notation
describe['trip_duration'] = describe['trip_duration'].apply(str)
describe['trip_duration'].loc[['mean', 'min', '25%', '50%', '75%', 'max']] = describe['trip_duration'].loc[['mean', 'min', '25%', '50%', '75%', 'max']].apply(seconds_to_pretty)
describe

We can see here that some values seem to be outliers:
* max trip_duration seems a way too long
* min trip_duration seems too short
* min passenger_count is 0

In [ ]:
# def pretty_print_max_trip_duration(df):   
#     max_trip_duration = df_train['trip_duration'].max()
#     print('Max duration trip: ({} sec) -> {} hours {} minutes {} secondes'.format(max_trip_duration, max_trip_duration // 3600, (max_trip_duration % 3600) // 60, (max_trip_duration % 3600) % 60))

# pretty_print_max_trip_duration(df_train)

let's make a boxplot to better outliers visualization

In [ ]:
# let's make a boxplot to better trip_duration outliers visualization
# we need to see if there is any correlation between trip_duration and store_and_fwd_flag
def boxplot_trip_duration(df, ylim=None):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
    plt.subplots_adjust(wspace=1)
    if ylim:
        ax.set_ylim(ylim)
    sns.boxplot(data=df_train, y='trip_duration', x='store_and_fwd_flag', fliersize=5, ax=ax).axes.set_title(label='Trip duration outliers visualization \n (store_and_fwd_flag = N)', fontsize=16, pad=25)
#     sns.boxplot(y=df_train[df_train['store_and_fwd_flag'] == 'N']['trip_duration'], fliersize=5, ax=ax1).axes.set_title(label='Trip duration outliers visualization \n (store_and_fwd_flag = N)', fontsize=16, pad=25)
#     sns.boxplot(y=df_train[df_train['store_and_fwd_flag'] == 'Y']['trip_duration'], fliersize=5, ax=ax2).axes.set_title(label='Trip duration outliers visualization \n (store_and_fwd_flag = Y)', fontsize=16, pad=25)
    
boxplot_trip_duration(df_train)

The higer trip_duration outliers seem to be present when store_and_fwd_flag = N, this is probably due to the vehicle did not have a connection to the server during a long time

In [ ]:
with_n = len(df_train[df_train['store_and_fwd_flag'] == 'N'].index)
with_y = len(df_train[df_train['store_and_fwd_flag'] == 'Y'].index)

len(df_train[df_train['store_and_fwd_flag'] == 'N'].index), len(df_train[df_train['store_and_fwd_flag'] == 'Y'].index)

There is a majority of rows with store_and_fwd_flag = no, so we can't afford to remove them

In [ ]:
seconds_to_pretty(df_train[df_train['store_and_fwd_flag'] == 'N']['trip_duration'].max()), seconds_to_pretty(df_train[df_train['store_and_fwd_flag'] == 'Y']['trip_duration'].max())

2 h 46 m 24 s may seem long for a trip to New York, but in doubt we will keep it

Let's zoom and the small values

In [ ]:
boxplot_trip_duration(df_train, [0, df_train[df_train['store_and_fwd_flag'] == 'N']['trip_duration'].mean()])

So we have to delete the values near 0

Let's see the passenger_count distribution

In [ ]:
_ = sns.distplot(df_train['passenger_count'], hist=False, rug=True)

There is a majority of trips with 1 passanger

Let's visualize the pickup positions


In [ ]:
_ = sns.jointplot(data=df_train, x="pickup_latitude", y="pickup_longitude")

Let's visualize the dropoff positions

In [ ]:
_ = sns.jointplot(data=df_train, x="dropoff_latitude", y="dropoff_longitude")

There is some outilers positions

# Load the test dataset

In [ ]:
df_test = pd.read_csv(os.path.join('..', 'input', 'test.csv'), index_col=0)
df_test.head()

In [ ]:
len(df_test.index)

# Compare train and test datasets

In [ ]:
df_train.describe().drop('count')

In [ ]:
df_test.describe().drop('count')

Latitudes and longitudes seems pretty similar

passenger_count seems pretty similar too, so we don't need to filter them

In [ ]:
_ = sns.distplot(df_train['passenger_count'], hist=False, rug=True).set_title(label='Train dataset', fontsize=16, pad=25)

In [ ]:
_ = sns.distplot(df_test['passenger_count'], hist=False, rug=True).set_title(label='Test dataset', fontsize=16, pad=25)

Distribution is pretty similar too

In [ ]:
_ = sns.jointplot(data=df_test, x="pickup_latitude", y="pickup_longitude")

In [ ]:
_ = sns.jointplot(data=df_test, x="dropoff_latitude", y="dropoff_longitude")

The test dataset seems to have more extreme coordinates

## Fix outliers

In first, let's delete all the rows with not stored and forwarded trip and trip_duration > max of trip_duration of rows with stored and forwarded trip

In [ ]:
max_trip_duration_flag_y = df_train[df_train['store_and_fwd_flag'] == 'Y']['trip_duration'].max()
row_count_before_deletion = len(df_train.index)
print('{} lines before deletion'.format(row_count_before_deletion))
df_train = df_train[~((df_train['store_and_fwd_flag'] == 'N') & (df_train['trip_duration'] > max_trip_duration_flag_y))]
row_count_after_deletion = len(df_train.index)
print('{} lines after deletion'.format(row_count_after_deletion))
print('{} lines deleted'.format(row_count_before_deletion - row_count_after_deletion))

We have deleted 2122 rows on 1458637 row, that's ok


let's check the outliers visualization now

In [ ]:
boxplot_trip_duration(df_train)

There seem to be still extreme values on Y column

In [ ]:
_ = sns.distplot(df_train[df_train['store_and_fwd_flag'] == 'Y']['trip_duration'], hist=False, rug=True)

In [ ]:
df_train[((df_train['store_and_fwd_flag'] == 'Y') & (df_train['trip_duration'] > 6500))]['trip_duration'].count()

~~I think we need to remove all trip durations > 6500~~, i think this method overfit the model :/

In [ ]:
# row_count_before_deletion = len(df_train.index)
# print('{} lines before deletion'.format(row_count_before_deletion))
# df_train = df_train[df_train['trip_duration'] <= 6500]
# row_count_after_deletion = len(df_train.index)
# print('{} lines after deletion'.format(row_count_after_deletion))
# print('{} lines deleted'.format(row_count_before_deletion - row_count_after_deletion))

In [ ]:
# _ = sns.distplot(df_train['trip_duration'], hist=False, rug=True)

Ok, we are good for the to long trip durations

1. Now let's remove the trip durations <= 10 sec, a trip duration <= 10 sec makes no sens

In [ ]:
row_count_before_deletion = len(df_train.index)
print('{} lines before deletion'.format(row_count_before_deletion))
df_train = df_train[df_train['trip_duration'] >= 10]
row_count_after_deletion = len(df_train.index)
print('{} lines after deletion'.format(row_count_after_deletion))
print('{} lines deleted'.format(row_count_before_deletion - row_count_after_deletion))

~~Now lets remove trips with passenger_count = 0~~

We dont have to remove passenger_count = 0 because the test dataset have too

In [ ]:
# df_train[df_train['passenger_count'] < 1]['passenger_count'].count()

In [ ]:
# row_count_before_deletion = len(df_train.index)
# print('{} lines before deletion'.format(row_count_before_deletion))
# df_train = df_train[df_train['passenger_count'] >= 1]
# row_count_after_deletion = len(df_train.index)
# print('{} lines after deletion'.format(row_count_after_deletion))
# print('{} lines deleted'.format(row_count_before_deletion - row_count_after_deletion))

# Scale the train dataset to the test dataset

Determine max and min lat, long from the test dataset

In [ ]:
# max_test_pickup_latitude = df_test['pickup_latitude'].max()
# min_test_pickup_latitude = df_test['pickup_latitude'].min()

# max_test_pickup_longitude = df_test['pickup_longitude'].max()
# min_test_pickup_longitude = df_test['pickup_longitude'].min()


# max_test_dropoff_latitude = df_test['dropoff_latitude'].max()
# min_test_dropoff_latitude = df_test['dropoff_latitude'].min()

# max_test_dropoff_longitude = df_test['dropoff_longitude'].max()
# min_test_dropoff_longitude = df_test['dropoff_longitude'].min()

# (max_test_pickup_latitude, min_test_pickup_latitude, '---',
#  max_test_pickup_longitude, min_test_pickup_longitude, '---',
#  max_test_dropoff_latitude, min_test_dropoff_latitude, '---',
#  max_test_dropoff_longitude, min_test_dropoff_longitude)

In [ ]:
# row_count_before_deletion = len(df_train.index)
# print('{} lines before deletion'.format(row_count_before_deletion))

# df_train['pickup_latitude'] = df_train[df_train['pickup_latitude'] <= max_test_pickup_latitude]['pickup_latitude']
# df_train['pickup_latitude'] = df_train[df_train['pickup_latitude'] >= min_test_pickup_latitude]['pickup_latitude']

# df_train['pickup_longitude'] = df_train[df_train['pickup_longitude'] <= max_test_pickup_longitude]['pickup_longitude']
# df_train['pickup_longitude'] = df_train[df_train['pickup_longitude'] >= min_test_pickup_longitude]['pickup_longitude']


# df_train['dropoff_latitude'] = df_train[df_train['dropoff_latitude'] <= max_test_dropoff_latitude]['dropoff_latitude']
# df_train['dropoff_latitude'] = df_train[df_train['dropoff_latitude'] >= min_test_dropoff_latitude]['dropoff_latitude']

# df_train['dropoff_longitude'] = df_train[df_train['dropoff_longitude'] <= max_test_dropoff_longitude]['dropoff_longitude']
# df_train['dropoff_longitude'] = df_train[df_train['dropoff_longitude'] >= min_test_dropoff_longitude]['dropoff_longitude']

# row_count_after_deletion = len(df_train.index)
# print('{} lines after deletion'.format(row_count_after_deletion))
# print('{} lines deleted'.format(row_count_before_deletion - row_count_after_deletion))

# Define data manipulation functions and variables

In [ ]:
from geopy.distance import geodesic

def create_datetime_based_columns(df):
    df['datetime'] =  pd.to_datetime(df['pickup_datetime'])
    
    df['year'] = df['datetime'].dt.year # year seems to help
    df['month'] = df['datetime'].dt.month # month doesn't help at all
    df['day'] = df['datetime'].dt.day
    df['dayofweek'] = df['datetime'].dt.weekday
    df['hour'] = df['datetime'].dt.hour
    
    return df

# Define filtering/split functions

In [ ]:
def filter_feature_columns(df):
    selected_columns = []
    selected_columns = ['pickup_longitude', 'pickup_latitude'] 
    selected_columns += ['dropoff_longitude', 'dropoff_latitude']
    selected_columns += ['month', 'dayofweek', 'hour']
#     selected_columns += ['store_and_fwd_flag']
#     selected_columns += ['lat_distance', 'long_distance']
    return df[selected_columns]

def filter_target_column(df):
    return df['trip_duration']
    

def filter_split_dataset(df):
    X = filter_feature_columns(df)
    y = filter_target_column(df)
    
    return X, y
 

# Let's measure loss

In [ ]:
df_train_copy = df_train.copy() # we have to work on a copy of df_train to be able to repeat the operations from 0 without having to reload the dataset
df_train_copy = create_datetime_based_columns(df_train_copy)
# tmp_df_train = normalize_store_and_fwd_flag(tmp_df_train)
# df_train_copy = create_distance_column(df_train_copy)

df_train_copy.head()

In [ ]:
X_train, y_train = filter_split_dataset(df_train_copy)
X_train.shape, y_train.shape

In [ ]:
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RandomizedSearchCV # test des combinaisons aléatoirent de parametres
from sklearn.model_selection import GridSearchCV # test toutes les combinaisons

Search to obtains the best RandomForestRegressor params

In [ ]:
# thi part take to much time, so I comment it
# rfr_gcv = RandomForestRegressor()
# param_distributions = {
#     'n_estimators' = [10, 100, 200, 300]
#     'min_samples_leaf' = [1, 5, 10]
#     'min_samples_split' = [2, 10, 15]
#     'max_depth' = [10, 40, 80, 90]
# }

# rs = GridSearchCV(rfr_gcv, param_distributions, scoring='neg_mean_squared_log_error')
# rs.fit(X_train, y_train)
# rs.best_params_

Lets estimate the score

In [ ]:
# rfr = RandomForestRegressor(n_estimators=19, min_samples_leaf=10, 
#                             min_samples_split=15, max_features='auto', max_depth=80, bootstrap=True)

# RandomForestRegressor with best_params
# rfr = RandomForestRegressor(n_estimators=300, min_samples_leaf=10, min_samples_split=15, 
#                             max_features='auto', max_depth=90, bootstrap=True) # that take too long time :/

rfr = RandomForestRegressor(n_estimators=30, min_samples_leaf=10, min_samples_split=15, 
                            max_features='auto', max_depth=90, bootstrap=True)

s_split = ShuffleSplit(n_splits=4, train_size=.12, test_size=.6) # allows to test on less data, so the cross validation takes less time

# I commented this line to improve kernel execution time
# np.sqrt(-cross_val_score(rfr, X_train, y_train, cv=s_split, scoring='neg_mean_squared_log_error', n_jobs=-1)).mean()

## Let's train the model with training values

In [ ]:
rfr.fit(X_train, y_train)

## Let's predict the test trip_duration values

In [ ]:
df_test_copy = df_test.copy()
df_test_copy = create_datetime_based_columns(df_test_copy)
# tmp_df_test = normalize_store_and_fwd_flag(tmp_df_test)
X_test = filter_feature_columns(df_test_copy)

y_test_pred = rfr.predict(X_test)
# 
submission = pd.DataFrame({'id': df_test.index, 'trip_duration': y_test_pred})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)
!ls